In [86]:
import sys
import os
import PIL
from PIL import Image
import psutil
import time

sys.path.append(os.path.abspath("../"))
from src.helper import rgb_to_cmyk, calculate_cmyk_percentage, predict_pdf, label_to_price

In [2]:
initial_cpu = psutil.cpu_percent(interval=None)
initial_memory = psutil.virtual_memory().used

img = Image.open("../outputs/pdfium_10dpi/page-001.jpg").convert("RGB")
final_cpu = psutil.cpu_percent(interval=None)
final_memory = psutil.virtual_memory().used

# Menghitung perubahan
cpu_usage = final_cpu - initial_cpu
memory_usage = final_memory - initial_memory

# print(f"Waktu eksekusi: {end_time - start_time:.4f} detik")
print(f"Penggunaan CPU: {cpu_usage:.2f}%")
print(f"Penggunaan memori: {memory_usage / (1024 ** 2):.2f} MB") 

Penggunaan CPU: 30.50%
Penggunaan memori: 0.77 MB


In [1]:
import pandas as pd

In [47]:
df = pd.read_csv("../datasets/cmyk_by_dpi_and_render_options.csv")
df

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff
0,0.392798,1,300,0.18,0.18,0.39,7.94,8.69,0.00,0.00
1,1.442968,2,300,1.31,1.06,0.01,3.64,6.02,0.00,0.00
2,0.197159,3,300,0.00,0.00,0.00,10.21,10.21,0.00,0.00
3,0.152346,4,300,0.00,0.00,0.00,11.03,11.03,0.00,0.00
4,0.151719,5,300,0.00,0.00,0.00,11.18,11.18,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
82207,0.020813,880,5,0.00,0.00,0.00,0.00,0.00,5.71,5.71
82208,0.028406,881,5,2.55,11.07,11.06,6.25,30.93,3.52,3.52
82209,0.003493,882,5,0.81,1.20,1.84,1.35,5.20,-0.09,0.09
82210,0.014516,883,5,4.67,27.89,29.58,12.44,74.58,-3.45,3.45


In [48]:
df.iloc[0:, :]

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff
3,0.152346,4,300,0.0,0.0,0.0,11.03,11.03,0.0,0.0


In [52]:
option_batch_count = df['dpi'].nunique() * df['page'].nunique()

In [57]:
df['options'] = None
option_batch_count = df['dpi'].nunique() * df['page'].nunique()
df.iloc[option_batch_count*0:option_batch_count*1, df.columns.get_loc('options')] = 'default'
df.iloc[option_batch_count*1:option_batch_count*2, df.columns.get_loc('options')] = 'txt_aa'
df.iloc[option_batch_count*2:option_batch_count*3, df.columns.get_loc('options')] = 'txt_img_aa'
df

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff,options
0,0.392798,1,300,0.18,0.18,0.39,7.94,8.69,0.00,0.00,default
1,1.442968,2,300,1.31,1.06,0.01,3.64,6.02,0.00,0.00,default
2,0.197159,3,300,0.00,0.00,0.00,10.21,10.21,0.00,0.00,default
3,0.152346,4,300,0.00,0.00,0.00,11.03,11.03,0.00,0.00,default
4,0.151719,5,300,0.00,0.00,0.00,11.18,11.18,0.00,0.00,default
...,...,...,...,...,...,...,...,...,...,...,...
82207,0.020813,880,5,0.00,0.00,0.00,0.00,0.00,5.71,5.71,txt_img_aa
82208,0.028406,881,5,2.55,11.07,11.06,6.25,30.93,3.52,3.52,txt_img_aa
82209,0.003493,882,5,0.81,1.20,1.84,1.35,5.20,-0.09,0.09,txt_img_aa
82210,0.014516,883,5,4.67,27.89,29.58,12.44,74.58,-3.45,3.45,txt_img_aa


In [63]:
df.to_csv("../datasets/cmyk_by_dpi_and_render_options.csv", index=False)

In [64]:
df = pd.read_csv("../datasets/cmyk_by_dpi_and_render_options.csv")

In [65]:
df

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff,options
0,0.392798,1,300,0.18,0.18,0.39,7.94,8.69,0.00,0.00,default
1,1.442968,2,300,1.31,1.06,0.01,3.64,6.02,0.00,0.00,default
2,0.197159,3,300,0.00,0.00,0.00,10.21,10.21,0.00,0.00,default
3,0.152346,4,300,0.00,0.00,0.00,11.03,11.03,0.00,0.00,default
4,0.151719,5,300,0.00,0.00,0.00,11.18,11.18,0.00,0.00,default
...,...,...,...,...,...,...,...,...,...,...,...
82207,0.020813,880,5,0.00,0.00,0.00,0.00,0.00,5.71,5.71,txt_img_aa
82208,0.028406,881,5,2.55,11.07,11.06,6.25,30.93,3.52,3.52,txt_img_aa
82209,0.003493,882,5,0.81,1.20,1.84,1.35,5.20,-0.09,0.09,txt_img_aa
82210,0.014516,883,5,4.67,27.89,29.58,12.44,74.58,-3.45,3.45,txt_img_aa


In [76]:
df[(df['dpi'] == 5) & (df['diff']>5)].pivot_table(index='options', values='diff', aggfunc='sum')

,diff
options,
default,1537.51
txt_aa,2200.90
txt_img_aa,2196.25


In [80]:
df[(df['dpi'] == 5) & (df['diff']>=50) & (df['options'] == 'default')]

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff,options
26522,0.017131,3,5,0.0,0.0,0.0,65.50,65.50,-55.29,55.29,default
26523,0.009017,4,5,0.0,0.0,0.0,75.99,75.99,-64.96,64.96,default
26524,0.009348,5,5,0.0,0.0,0.0,75.99,75.99,-64.81,64.81,default
26525,0.009172,6,5,0.0,0.0,0.0,75.99,75.99,-65.12,65.12,default
26526,0.009151,7,5,0.0,0.0,0.0,75.99,75.99,-64.75,64.75,default
26527,0.009030,8,5,0.0,0.0,0.0,75.99,75.99,-65.06,65.06,default
26528,0.011086,9,5,0.0,0.0,0.0,75.99,75.99,-65.01,65.01,default
26529,0.008791,10,5,0.0,0.0,0.0,75.99,75.99,-65.08,65.08,default
26530,0.009189,11,5,0.0,0.0,0.0,75.99,75.99,-64.69,64.69,default
26531,0.008921,12,5,0.0,0.0,0.0,75.99,75.99,-65.00,65.00,default


In [83]:
import pypdfium2 as pdfium
pdf = pdfium.PdfDocument("../datasets/statistik-indonesia-2024-combined.pdf")

In [151]:
bitmap = pdf[0].render(scale=1/72 * 20, optimize_mode='print')
calculate_cmyk_percentage(bitmap.to_pil())

(0.17, 0.18, 0.43, 8.68)

In [167]:
len(df[(df['dpi']==40) & (df['diff']<5) & (df['options'] == 'default')]) == 884

False

In [168]:
df[(df['dpi']==40) & (df['diff']>=4) & (df['options'] == 'default')]

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff,options
22986,0.295964,3,40,0.0,0.0,0.0,14.56,14.56,-4.35,4.35,default
22987,0.013837,4,40,0.0,0.0,0.0,16.11,16.11,-5.08,5.08,default
22988,0.014283,5,40,0.0,0.0,0.0,16.27,16.27,-5.09,5.09,default
22989,0.013793,6,40,0.0,0.0,0.0,15.93,15.93,-5.06,5.06,default
22990,0.014056,7,40,0.0,0.0,0.0,16.33,16.33,-5.09,5.09,default
22991,0.013799,8,40,0.0,0.0,0.0,16.02,16.02,-5.09,5.09,default
22992,0.016063,9,40,0.0,0.0,0.0,16.06,16.06,-5.08,5.08,default
22993,0.013681,10,40,0.0,0.0,0.0,15.99,15.99,-5.08,5.08,default
22994,0.014132,11,40,0.0,0.0,0.0,16.40,16.40,-5.10,5.10,default
22995,0.013711,12,40,0.0,0.0,0.0,16.07,16.07,-5.08,5.08,default


In [1]:
import os

In [3]:
ROOT_DIR = os.path.abspath("../")
ROOT_DIR

'/Users/rasyidev/Documents/rasyidevcode/print-cost'

In [4]:
os.getcwd()

'/Users/rasyidev/Documents/rasyidevcode/print-cost/notebook'

In [5]:
os.path.(os.getcwd(), "../")

'/Users/rasyidev/Documents/rasyidevcode/print-cost/notebook/../'

In [17]:
from pathlib import Path

In [18]:
ROOT_DIR = Path().resolve().parent

In [19]:
current_dir

PosixPath('/Users/rasyidev/Documents/rasyidevcode/print-cost')

In [20]:
import pandas as pd

df = pd.read_csv(current_dir.joinpath("datasets/cmyk_by_dpi_and_render_options.csv"))
df

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff,options
0,0.392798,1,300,0.18,0.18,0.39,7.94,8.69,0.00,0.00,default
1,1.442968,2,300,1.31,1.06,0.01,3.64,6.02,0.00,0.00,default
2,0.197159,3,300,0.00,0.00,0.00,10.21,10.21,0.00,0.00,default
3,0.152346,4,300,0.00,0.00,0.00,11.03,11.03,0.00,0.00,default
4,0.151719,5,300,0.00,0.00,0.00,11.18,11.18,0.00,0.00,default
...,...,...,...,...,...,...,...,...,...,...,...
82207,0.020813,880,5,0.00,0.00,0.00,0.00,0.00,5.71,5.71,txt_img_aa
82208,0.028406,881,5,2.55,11.07,11.06,6.25,30.93,3.52,3.52,txt_img_aa
82209,0.003493,882,5,0.81,1.20,1.84,1.35,5.20,-0.09,0.09,txt_img_aa
82210,0.014516,883,5,4.67,27.89,29.58,12.44,74.58,-3.45,3.45,txt_img_aa


In [21]:
df_default = df[df['options'] == 'default'].copy()
df_default

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff,options
0,0.392798,1,300,0.18,0.18,0.39,7.94,8.69,0.00,0.00,default
1,1.442968,2,300,1.31,1.06,0.01,3.64,6.02,0.00,0.00,default
2,0.197159,3,300,0.00,0.00,0.00,10.21,10.21,0.00,0.00,default
3,0.152346,4,300,0.00,0.00,0.00,11.03,11.03,0.00,0.00,default
4,0.151719,5,300,0.00,0.00,0.00,11.18,11.18,0.00,0.00,default
...,...,...,...,...,...,...,...,...,...,...,...
27399,0.023905,880,5,0.00,0.00,0.00,6.24,6.24,-0.52,0.52,default
27400,0.032836,881,5,2.55,11.07,11.06,15.09,39.77,-5.30,5.30,default
27401,0.003468,882,5,0.81,1.20,1.84,1.35,5.20,-0.09,0.09,default
27402,0.016195,883,5,4.67,27.89,29.58,16.01,78.15,-7.01,7.01,default


In [24]:
df_default[df_default['dpi'] == 50].describe()

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff
count,884.000000,884.000000,884.0,884.000000,884.000000,884.000000,884.000000,884.000000,884.000000,884.000000
mean,0.049277,442.500000,50.0,2.796290,0.208507,7.248020,6.234706,16.487523,-0.055611,0.200385
std,0.179963,255.333116,0.0,3.965736,1.488747,5.662478,3.485683,12.689008,0.525763,0.489207
min,0.003323,1.000000,50.0,0.000000,0.000000,0.000000,0.060000,0.060000,-3.060000,0.000000
25%,0.028637,221.750000,50.0,1.690000,0.020000,4.712500,4.950000,11.692500,-0.090000,0.040000
50%,0.033334,442.500000,50.0,2.715000,0.040000,7.605000,5.960000,16.315000,0.025000,0.090000
75%,0.039224,663.250000,50.0,3.350000,0.060000,9.417500,6.860000,19.550000,0.100000,0.180000
max,4.986121,884.000000,50.0,60.500000,26.670000,52.090000,46.890000,157.750000,1.400000,3.060000


In [27]:
df_default[(df_default['dpi'] == 5) & (df_default['diff'] >=10)]

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff,options
26522,0.017131,3,5,0.00,0.00,0.00,65.50,65.50,-55.29,55.29,default
26523,0.009017,4,5,0.00,0.00,0.00,75.99,75.99,-64.96,64.96,default
26524,0.009348,5,5,0.00,0.00,0.00,75.99,75.99,-64.81,64.81,default
26525,0.009172,6,5,0.00,0.00,0.00,75.99,75.99,-65.12,65.12,default
26526,0.009151,7,5,0.00,0.00,0.00,75.99,75.99,-64.75,64.75,default
26527,0.009030,8,5,0.00,0.00,0.00,75.99,75.99,-65.06,65.06,default
26528,0.011086,9,5,0.00,0.00,0.00,75.99,75.99,-65.01,65.01,default
26529,0.008791,10,5,0.00,0.00,0.00,75.99,75.99,-65.08,65.08,default
26530,0.009189,11,5,0.00,0.00,0.00,75.99,75.99,-64.69,64.69,default
26531,0.008921,12,5,0.00,0.00,0.00,75.99,75.99,-65.00,65.00,default


In [29]:
df[(df['dpi'] == 5) & (df['diff'] >=10) & (df['options'] == 'txt_img_aa' )]

,converting_time,page,dpi,c,m,y,k,sum,base_minus_curr_sum,diff,options
81330,0.017049,3,5,0.00,0.00,0.00,64.79,64.79,-54.63,54.63,txt_img_aa
81331,0.008846,4,5,0.00,0.00,0.00,75.99,75.99,-65.02,65.02,txt_img_aa
81332,0.009185,5,5,0.00,0.00,0.00,75.99,75.99,-64.87,64.87,txt_img_aa
81333,0.009072,6,5,0.00,0.00,0.00,75.99,75.99,-65.19,65.19,txt_img_aa
81334,0.008983,7,5,0.00,0.00,0.00,75.99,75.99,-64.81,64.81,txt_img_aa
81335,0.008867,8,5,0.00,0.00,0.00,75.99,75.99,-65.11,65.11,txt_img_aa
81336,0.011102,9,5,0.00,0.00,0.00,75.99,75.99,-65.07,65.07,txt_img_aa
81337,0.008678,10,5,0.00,0.00,0.00,75.99,75.99,-65.15,65.15,txt_img_aa
81338,0.009008,11,5,0.00,0.00,0.00,75.99,75.99,-64.78,64.78,txt_img_aa
81339,0.008721,12,5,0.00,0.00,0.00,75.99,75.99,-65.06,65.06,txt_img_aa


## IDEA
- Bikin model untuk setiap dpi 300, 50, 5
- Kalo 300, pakai fitur sum
- kalo 50, 5, pakai fitur cmyk dan sum
- bandingin hasil 3 ini dengan 300 dpi

In [32]:
df2 = pd.read_csv("../outputs/dpi_300_50_5_accuracy.csv")
df2

,filename,page,res300,res50,res5
0,How Charts Lie Getting Smarter about Visual In...,1,2000,2000,2000
1,How Charts Lie Getting Smarter about Visual In...,2,500,500,500
2,How Charts Lie Getting Smarter about Visual In...,3,500,500,500
3,How Charts Lie Getting Smarter about Visual In...,4,500,500,500
4,How Charts Lie Getting Smarter about Visual In...,5,500,500,500
...,...,...,...,...,...
730,Mackay H. Python Programming. Handbook For IoT...,135,500,500,500
731,Mackay H. Python Programming. Handbook For IoT...,136,500,500,500
732,Mackay H. Python Programming. Handbook For IoT...,137,500,500,500
733,Mackay H. Python Programming. Handbook For IoT...,138,500,500,500


In [34]:
df2[(df2['res300'] == df2['res50']) & (df2['res50'] == df2['res5'])]

,filename,page,res300,res50,res5
0,How Charts Lie Getting Smarter about Visual In...,1,2000,2000,2000
1,How Charts Lie Getting Smarter about Visual In...,2,500,500,500
2,How Charts Lie Getting Smarter about Visual In...,3,500,500,500
3,How Charts Lie Getting Smarter about Visual In...,4,500,500,500
4,How Charts Lie Getting Smarter about Visual In...,5,500,500,500
...,...,...,...,...,...
730,Mackay H. Python Programming. Handbook For IoT...,135,500,500,500
731,Mackay H. Python Programming. Handbook For IoT...,136,500,500,500
732,Mackay H. Python Programming. Handbook For IoT...,137,500,500,500
733,Mackay H. Python Programming. Handbook For IoT...,138,500,500,500
